In [19]:
###IMPORTANT ####
 # Actor until 1976
 # Actor in a Leading Role since 1977
 # Same for Actress
    
### Stock the data better than doing a request every try # Done

### PROBLEME : COMMENT GERER LES 2 WINNERS, existe plusieurs fois # Done

### PROBLEM : for DIRECTION, names and titles are swapped # Done


### PROBLEME : Incohérence des noms de catégories sur plusieurs année : best motion picture -> best picture
# Special effect -> Visual effects etc

### Les informations récoltées ne sont pas les mêmes ex : foreign film c'est le pays qui est récolté, comment faire ?



import sys
!{sys.executable} -m pip install jellyfish

In [158]:
### OOP   ### ROTTENTOMATOES

import time
import requests
from scrapy import Selector
from tqdm import tqdm
import pandas as pd
import re  # text processing
import numpy as np
import json
import jellyfish
from urllib.parse import quote

class Oscar_Scraper:
    
    list_names = []
    list_year = []
    list_category = []
    list_films = []
    list_results = [] 
    list_ratings_imdb = []
    list_ratings_rottentomatoes = []
    list_metacritics = []
    list_boxoffice = []
    list_id_movie = []
    list_birthday = []
    list_gender = []
    list_id_indiv = []
    list_names_right = []
    list_films_right = []
    list_names_wrong = []
    list_films_wrong = []
    
    def __init__(self, from_time, to_time):
        self.links = {
            'oscars':[],
            'rotten':[],
        }
        
        self.categories_interest = ['Directing', 'Actor', 'Actor in a Leading Role', 'Actress', 'Actress in a Leading Role',
                  'Outstanding Picture','Outstanding Production','Best Motion Picture', 'Best Picture', 'Foreign Language Film', 
                  'Special Effects', 'Special Visual Effects','Special Achievement Award (Visual Effects)', 'Visual Effects']
        
        self.categories_individuals = ['Directing', 'Actor', 'Actor in a Leading Role', 'Actress', 'Actress in a Leading Role']
        self.categories_films = ['Outstanding Picture','Outstanding Production','Best Motion Picture', 'Best Picture', 'Foreign Language Film', 
                  'Special Effects', 'Special Visual Effects','Special Achievement Award (Visual Effects)', 'Visual Effects']
        
        #lists of different names for same categories
        self.categories_dictionary = {
                    "Actor": ['Actor', 'Actor in a Leading Role'],
                    "Actress": ['Actress', 'Actress in a Leading Role'],
                    "Picture": ['Outstanding Picture', 'Outstanding Production','Best Motion Picture', 'Best Picture'],
                    "VE" : ['Special Effects', 'Special Visual Effects','Special Achievement Award (Visual Effects)','Visual Effects']
                    }
        
        
        self.from_time = from_time
        self.time_period = range(from_time, to_time + 1)
        self.data = {
            'category':[],
            'film':[],
            'year':[],
            'name':[],
            'gender':[],
            'result':[],
            'birthdate':[],
            'imbd_ratings':[],
            'rotten_ratings':[],
            'metacritics_ratings':[],
            'box office':[],
            'id_movie':[]
            
        }
        
        

    def getHTML(self):
        
        for i in self.time_period:
            #time.sleep(10) # Crawlerdelay
            url = "https://www.oscars.org/oscars/ceremonies/{}".format(i)
            response_oscars = requests.get(url)
            sel = Selector(text = response_oscars.text)   # Scrapy chose over BeautifulSoup for selector CSS
            self.links['oscars'].append(sel)

    def getNominees(self, years, i, category, number_nominees, number_winners):

        for number in range(number_nominees):   
            if number_winners == 1:
                # NAME
                path_name = "#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div:nth-child({}) > div.views-field.views-field-field-actor-name > h4::text".format(i+1, 4 +number)
                nominee_name = ''.join(self.links['oscars'][years].css(path_name).extract()) # ''.join() transforms the list produced by the selector into a string
            if number_winners == 2:
                 # NAME
                path_name = "#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div:nth-child({}) > div.views-field.views-field-field-actor-name > h4::text".format(i+1, 5 +number)
                nominee_name = ''.join(self.links['oscars'][years].css(path_name).extract()) # ''.join() transforms the list produced by the selector into a string
            
            if category == "Directing": # For Directing, titles and names are swapped on the website
                self.list_films.append(nominee_name)
            else:
                self.list_names.append(nominee_name)

            # FILM
            if number_winners == 1:
                path_film = "#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div:nth-child({}) > div.views-field.views-field-title > span::text".format(i+1, 4 +number)
                nominee_film = ''.join(self.links['oscars'][years].css(path_film).extract()) # ''.join() transforms the list produced by the selector into a string
                nominee_film = nominee_film.replace("\n", "") # Cleaning up the string
            if number_winners == 2:
                path_film = "#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div:nth-child({}) > div.views-field.views-field-title > span::text".format(i+1, 5 +number)
                nominee_film = ''.join(self.links['oscars'][years].css(path_film).extract()) # ''.join() transforms the list produced by the selector into a string
                nominee_film = nominee_film.replace("\n", "") # Cleaning up the string
            if category == "Directing":# For Directing, titles and names are swapped on the website
                self.list_names.append(nominee_film)
            else:
                self.list_films.append(nominee_film)
            
            self.list_year.append(self.from_time + years)         
            self.list_category.append(category)
            self.list_results.append("Nominee")

    def getWinners(self, years, i, category, number_winners):

        if number_winners == 1: # 1 Winner
        # NAME
            path_name = "#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div.views-row.views-row-1.views-row-odd.views-row-first.views-row-last > div.views-field.views-field-field-actor-name > h4::text".format(i+1)
            nominee_name = ''.join(self.links['oscars'][years].css(path_name).extract()) # ''.join() transforms the list produced by the selector into a string

            if category == "Directing": # For Directing, titles and names are swapped on the website
                self.list_films.append(nominee_name)
            else:
                self.list_names.append(nominee_name)
            
        
            # FIlM
            path_film = '#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div.views-row.views-row-1.views-row-odd.views-row-first.views-row-last > div.views-field.views-field-title > span::text'.format(i+1)
            nominee_film = ''.join(self.links['oscars'][years].css(path_film).extract()) # ''.join() transforms the list produced by the selector into a string
            nominee_film = nominee_film.replace("\n", "") # Cleaning up the string

            if category == "Directing":# For Directing, titles and names are swapped on the website
                self.list_names.append(nominee_film)
            else:
                self.list_films.append(nominee_film)
           
            self.list_year.append(self.from_time + years)
            self.list_category.append(category)
            self.list_results.append("Winner")


        if number_winners == 2: # 2 Winners
            # NAME 1st Winner 
            path_name = "#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div:nth-child(2) > div.views-field.views-field-field-actor-name > h4::text".format(i+1)
            nominee_name = ''.join(self.links['oscars'][years].css(path_name).extract()) # ''.join() transforms the list produced by the selector into a string4

            if category == "Directing": # For Directing, titles and names are swapped on the website
                self.list_films.append(nominee_name)
            else:
                self.list_names.append(nominee_name)

            self.list_year.append(self.from_time + years)
            self.list_category.append(category)
            self.list_results.append("Tie-winner")

            # NAME 2st Winner 
            path_name = "#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div:nth-child(3) > div.views-field.views-field-field-actor-name > h4::text".format(i+1)
            nominee_name = ''.join(self.links['oscars'][years].css(path_name).extract()) # ''.join() transforms the list produced by the selector into a string4

            if category == "Directing": # For Directing, titles and names are swapped on the website
                self.list_films.append(nominee_name)
            else:
                self.list_names.append(nominee_name)

            self.list_year.append(self.from_time + years)
            self.list_category.append(category)
            self.list_results.append("Tie-winner")

            # FILM
            path_film1 = '#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div:nth-child(2) > div.views-field.views-field-title > span::text'.format(i+1)
            path_film2 = '#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div:nth-child(3) > div.views-field.views-field-title > span::text'.format(i+1)

            nominee_film = ''.join(self.links['oscars'][years].css(path_film1).extract()) # ''.join() transforms the list produced by the selector into a string
            nominee_film = nominee_film.replace("\n", "") # Cleaning up the string

            if category == "Directing":# For Directing, titles and names are swapped on the website
                self.list_names.append(nominee_film)
            else:
                self.list_films.append(nominee_film)

            winner2_film = ''.join(self.links['oscars'][years].css(path_film2).extract()) # ''.join() transforms the list produced by the selector into a string
            winner2_film = winner2_film.replace("\n", "") # Cleaning up the string

            if category == "Directing":# For Directing, titles and names are swapped on the website
                self.list_names.append(winner2_film)
            else:
                self.list_films.append(winner2_film)        



    def getDATA(self):
            
        for years in tqdm(range(len(self.links['oscars']))):

            categories = self.links['oscars'][years].css('#quicktabs-tabpage-honorees-0 > div > div.view-content > div.view-grouping > div.view-grouping-header > h2::text').extract()    
            
            for i in range(len(categories)):
                if categories[i] in self.categories_individuals:

                    # Need the number of winners (might be ties) and nominated          
                    path_nominated = "#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div:last-child".format(i+1)
                    number_of_film_nominated = int(''.join(self.links['oscars'][years].css(path_nominated).extract()).split("views-row-")[1])

                    path_winner = "#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div:nth-child(2)".format(i+1)
                    trial = ''.join(self.links['oscars'][years].css(path_winner).extract()).split("views-row-first views-row-last") # If 1 winner then it splits the string into 1 list of 2 elements, if 2 winners, it does nothing
                    if len(trial) == 2:
                        number_winners = 1
                    elif len(trial) == 1:
                        number_winners = 2
                    else:
                        print("problem")


                    ## WINNERS                 
                    self.getWinners(years, i, categories[i], number_winners)     
                    ## NOMINEES
                    self.getNominees(years, i, categories[i], number_of_film_nominated, number_winners)   
                    
                    
        self.data = {
            'year': self.list_year,
            'category': self.list_category,
            'film': self.list_films,
            'name': self.list_names,
            #'gender':[],
            'result':self.list_results
            #,
            #'birthdate':[]
        }            
        
        list_films_right = pd.read_csv("list_films_right.csv")
        list_films_wrong = pd.read_csv("list_films_wrong.csv")
        list_names_right = pd.read_csv("list_names_right.csv")
        list_names_wrong = pd.read_csv("list_names_wrong.csv")
        l_films = len(list_films_right)
        l_names = len(list_names_right)
        l_data = len(self.data["film"])
        
        for i in range(max(l_films,l_names)):
            if i < l_names:
                self.list_names_right.append(list_names_right.values.tolist()[i][0])
                self.list_names_wrong.append(list_names_wrong.values.tolist()[i][0])
            if i < l_films:
                self.list_films_right.append(list_films_right.values.tolist()[i][0])
                self.list_films_wrong.append(list_films_wrong.values.tolist()[i][0])
                
        for i in range(l_data):
            if self.data["film"][i] == "Red":
                self.data["film"][i] = "Three Colors: Red"
            for y in range(max(l_films,l_names)):
                if y < l_films:
                    if self.data['film'][i] == self.list_films_wrong[y]:
                        self.data['film'][i] = self.list_films_right[y]
                if y < l_names:       
                    if self.data['name'][i] == self.list_names_wrong[y]:
                        self.data['name'][i] = self.list_names_right[y]

    def getINDIVIDUALS(self):
        df = pd.DataFrame(self.data)
        df = df.set_index(['year','category'])
        individuals = df.loc(axis = 0)[pd.IndexSlice[:, self.categories_individuals]]
        return(individuals)
    
    def getFILMS(self):
        df = pd.DataFrame(self.data)
        df = df.set_index(['year','category'])
        films = df.loc(axis = 0)[pd.IndexSlice[:, self.categories_interest[5:]]]
        films.columns = ('film', 'Studio/Creator(s)', 'result')
        return(films)
    
     #creating big dataframe with all data
    def getAllCategories(self):
        df = pd.DataFrame(self.data)
        df = df.set_index(['year','category'])
        return(df)
    
    #in progress
    """ #creating CSV files based on scraped data. Possible naming of the columns for different datasets
    def create_named_CSV(self,dataframe,categories,columns_name):
        if len(column_name) > 2:
          raise Exception('Items exceeds the maximum allowed length of 2')
        category = []
        if type(categories) == 'str':
        category.append(categories)
        for x in self.categories_dictionary:
            for y in range(len(self.categories_dictionary[x])):
                if category == self.categories_dictionary[x][y]:
                    if x not in names:
                        names.append(x)
        for x in names:
            create_empty_CSV(x)
        df_final.columns = columns_name
             films.to_csv(i + '.csv', sep=' ', mode='a')"""
        
    #creating empty csv names after argument of the method   
    def create_empty_CSV(*stringcsv_names):
        for i in stringcsv_names:
            with open(i + '.csv', 'w') as f:
                pass
            
######################################################### 
    
    def getAPI_omdb(self):
        key = "59d5666b"
        url_api = "https://www.omdbapi.com/?t={}&y={}&apikey={}"
        for film_number in tqdm(range(len(self.data['film']))):
            film = self.data['film']
            title_standard = film[film_number].upper().replace(" ","+").replace(".","").replace("&","%26").replace("ONCE+UPON+A+TIMEIN+HOLLYWOOD", "ONCE+UPON+A+TIME...+IN+HOLLYWOOD")
            year_standard = self.data['year'][film_number]-1
            response_api = json.loads(requests.get(url_api.format(title_standard, year_standard, key)).text)
            if response_api["Response"] != "False":        
                self.Ombd_get(response_api)
            else:
                year_standard = self.data['year'][film_number]-2
                response_api = json.loads(requests.get(url_api.format(title_standard, year_standard, key)).text)
                if response_api["Response"] != "False":               
                    self.Ombd_get(response_api)
                else:
                    year_standard = self.data['year'][film_number]-3
                    response_api = json.loads(requests.get(url_api.format(title_standard, year_standard, key)).text)
                    if response_api["Response"] != "False":
                        self.Ombd_get(response_api)
                    else:
                        year_standard = self.data['year'][film_number]
                        response_api = json.loads(requests.get(url_api.format(title_standard, year_standard, key)).text)
                        if response_api["Response"] != "False":
                            self.Ombd_get(response_api)
                        else:
                            title_standard = title_standard.replace("%26","and")
                            year_standard = self.data['year'][film_number]-1
                            response_api = json.loads(requests.get(url_api.format(title_standard, year_standard, key)).text)
                            if response_api["Response"] != "False":
                                self.Ombd_get(response_api)
                            else:
                                year_standard = self.data['year'][film_number]-2
                                response_api = json.loads(requests.get(url_api.format(title_standard, year_standard, key)).text)
                                if response_api["Response"] != "False":
                                    self.Ombd_get(response_api)
                                else: 
                                    year_standard = self.data['year'][film_number]-3
                                    response_api = json.loads(requests.get(url_api.format(title_standard, year_standard, key)).text)
                                    if response_api["Response"] != "False":
                                        self.Ombd_get(response_api)
                                    else:
                                        print("Error",film[film_number],title_standard,year_standard)
            
        # Standardization
        
        for i in tqdm(range(len(self.data['film']))): 
            if self.list_boxoffice[i] == "N/A":
                self.list_boxoffice[i] = np.nan
            else:
                self.list_boxoffice[i] = float(self.list_boxoffice[i].replace("$","").replace(",",""))

            if self.list_ratings_imdb[i] == "N/A":
                self.list_ratings_imdb[i] = np.nan
            else:
                self.list_ratings_imdb[i] = float(self.list_ratings_imdb[i])

            if self.list_ratings_rottentomatoes[i] == "N/A":
                self.list_ratings_rottentomatoes[i] = np.nan
            else:
                self.list_ratings_rottentomatoes[i] = float(self.list_ratings_rottentomatoes[i].replace("%",""))

            if self.list_metacritics[i] == "N/A":
                self.list_metacritics[i] = np.nan
            else:
                self.list_metacritics[i] = float(self.list_metacritics[i].replace("/100",""))
            if self.data["film"][i] == "The Postman (Il Postino)":
                self.list_id_movie[i] = "tt0110877"
    
        self.data['imbd_ratings'] = self.list_ratings_imdb,
        self.data['rotten_ratings'] = self.list_ratings_rottentomatoes,
        self.data['metacritics_ratings'] = self.list_metacritics,
        self.data['box office'] = self.list_boxoffice
        self.data['id_movie'] = self.list_id_movie
        
        
        
    def Ombd_get(self, response_api):
        for ratings in range(len(response_api["Ratings"])):

            if response_api["Ratings"][ratings]["Source"] == "Rotten Tomatoes":
                self.list_ratings_rottentomatoes.append(response_api["Ratings"][ratings]["Value"])

            if response_api["Ratings"][ratings]["Source"] == "Metacritic":
                self.list_metacritics.append(response_api["Ratings"][ratings]["Value"]) 
                    
        if not any("Rotten Tomatoes" in response_api["Ratings"][ratings]["Source"] for ratings in range(len(response_api["Ratings"]))):
            self.list_ratings_rottentomatoes.append("N/A")
        if not any("Metacritic" in response_api["Ratings"][ratings]["Source"] for ratings in range(len(response_api["Ratings"]))):
            self.list_metacritics.append("N/A")

        self.list_ratings_imdb.append(response_api.get("imdbRating", "N/A"))            
        self.list_boxoffice.append(response_api.get("BoxOffice", "N/A"))
        self.list_id_movie.append(response_api.get("imdbID", "N/A"))            

        
        
    def getAPI_omdb(self):
        API_KEY_MDB = "a68690ebf69567801e68c26ee82d7787"
        url_MDB_search = "https://api.themoviedb.org/3/search/movie?api_key={}&language=en-US&query={}&page={}&include_adult=false"
        url_MDB_person = "https://api.themoviedb.org/3/person/{}?api_key={}"
        for film_number in tqdm(range(len(self.data['film']))):
            title_standard = quote(self.data['film'][film_number])
            response_search = json.loads(requests.get(url_MDB_search.format(API_KEY_MDB,title_standard,1)).text)
            if response_search["total_results"] != 0:        
                if self.TMDB_get(response_search, film_number, 0) == 1:

                    response_person = json.loads(requests.get(url_MDB_person.format(self.list_id_indiv[film_number],API_KEY_MDB)).text)
                    if response_person.get("success") != False: # Check if the API find a person

                        if response_person["birthday"] in (None,0):
                            pass
                        
                        if response_person["gender"] in (None,0):
                            pass
                        self.list_birthday.append(response_person["birthday"])
                        self.list_gender.append(response_person["gender"])
                    else:
                        pass
                else:
                    response_search = json.loads(requests.get(url_MDB_search.format(API_KEY_MDB,title_standard,2)).text)
                    if response_search["total_results"] != 0:        
                        if self.TMDB_get(response_search, film_number, 1) == 1:
                            response_person = json.loads(requests.get(url_MDB_person.format(self.list_id_indiv[film_number],API_KEY_MDB)).text)
                            if response_person.get("success") != False: # Check if the API find a person
                                if response_person["birthday"] in (None,0):
                                    pass
                                
                                if response_person["gender"] in (None,0):
                                    pass
                                self.list_birthday.append(response_person["birthday"])
                                self.list_gender.append(response_person["gender"])
                            else:
                                pass
                        else:
                            # problem with some 1930's movies
                            url_MDB_search1930 = "https://api.themoviedb.org/3/search/movie?api_key={}&language=en-US&query={}&page={}&include_adult=false&year=1930"
                            response_search = json.loads(requests.get(url_MDB_search1930.format(API_KEY_MDB,title_standard,1)).text)
                            if response_search["total_results"] != 0:        
                                if self.TMDB_get(response_search, film_number, 1) == 1:
                                    response_person = json.loads(requests.get(url_MDB_person.format(self.list_id_indiv[film_number],API_KEY_MDB)).text)
                                    if response_person.get("success") != False: # Check if the API find a person
                                        
                                        if response_person["birthday"] in (None,0):
                                            pass
                                        if response_person["gender"] in (None,0):
                                            pass
                                        self.list_birthday.append(response_person["birthday"])
                                        self.list_gender.append(response_person["gender"])
                                    else:
                                        pass
            else:
                pass
                self.Correction(film_number)
        
        for i in tqdm(range(len(self.data['film']))): 
            if self.list_gender[i] == 1:
                self.list_gender[i] = "Female"
            elif self.list_gender[i] == 2:
                self.list_gender[i] = "Male"
            else:
                pass
##
            self.list_birthday[i] = int(self.list_birthday[i][0:4]) # We keep only the year and transform from string to float
            
        self.data['gender'] = self.list_gender,
        self.data['birthday'] = self.list_birthday
        
        
                                        
    def TMDB_get(self, response_search, film_number, number_of_time):
        url_MDB_credit = "http://api.tmdb.org/3/movie/{}/credits?api_key={}"
        API_KEY_MDB = "a68690ebf69567801e68c26ee82d7787"
        count = 0
        
        for number_results in range(len(response_search["results"])):
            if count == 0:
                if response_search["results"][number_results].get("release_date") not in (None,0,''): # There is a release date
                    if (int(response_search["results"][number_results]["release_date"][0:4]) in (self.data["year"][film_number],self.data["year"][film_number]-1, self.data["year"][film_number]-2, self.data["year"][film_number]-3)) and count == 0:
                        id_MDB = response_search["results"][number_results]["id"]
                        response_credit = json.loads(requests.get(url_MDB_credit.format(id_MDB,API_KEY_MDB)).text)
                        if response_credit.get("success") != False: # Check if the API find a person
                            if self.data["category"][film_number] == "Directing":
                                for acteurs in range(len(response_credit["crew"])):
                                    if jellyfish.damerau_levenshtein_distance(response_credit["crew"][acteurs]["name"], self.data["name"][film_number]) < 2:
                                        self.list_id_indiv.append(response_credit["crew"][acteurs]["id"])
                                        count = count + 1
                                       
                                        break


                                if not any(jellyfish.damerau_levenshtein_distance(response_credit["crew"][acteurs]["name"], self.data["name"][film_number]) < 2 for acteurs in range(len(response_credit["crew"]))):
                                    pass
                                    # self.Correction(film_number)
                                    
                            else:
                                for acteurs in range(len(response_credit["cast"])):
                                    if jellyfish.damerau_levenshtein_distance(response_credit["cast"][acteurs]["name"], self.data["name"][film_number]) < 2:
                                        self.list_id_indiv.append(response_credit["cast"][acteurs]["id"])
                                        count = count + 1 # Empêche d'avoir plusieurs fois le même acteur si on l'a déjà trouvé
                                        break   
                                        
                                if not any(jellyfish.damerau_levenshtein_distance(response_credit["cast"][acteurs]["name"], self.data["name"][film_number]) < 2 for acteurs in range(len(response_credit["cast"]))):
                                    pass
                                    # self.Correction(film_number)
                                    

                        else:
                            pass
                    else:
                        pass

                else:
                    pass
        return(count)
                                
         # Standardization
        
    #    

    def Correction(self,film_number):
        
        question = input("Film (f) or Name (n) or Both (b) or Pass (p)?")
        if question == "f":
            answer = input("Bon titre film ?")
            self.list_films_wrong.append(self.data["film"][film_number])
            self.list_films_right.append(answer)
            self.data["film"][film_number] = answer
            pd.DataFrame(self.list_films_right).to_csv('list_films_right.csv',index=False) 
            pd.DataFrame(self.list_films_wrong).to_csv('list_films_wrong.csv',index=False) 
        if question == "n":
            answer = input("Bon nom ?")
            self.list_names_wrong.append(self.data["name"][film_number])
            self.list_names_right.append(answer)           
            self.data["name"][film_number] = answer
            pd.DataFrame(self.list_names_right).to_csv('list_names_right.csv',index=False) 
            pd.DataFrame(self.list_names_wrong).to_csv('list_names_wrong.csv',index=False) 
        if question == "b":
            answer1 = input("Bon titre film ?")
            answer2 = input("Bon nom ?")
            self.list_films_wrong.append(self.data["film"][film_number])
            self.list_names_wrong.append(self.data["name"][film_number])
            self.list_films_right.append(answer1)
            self.list_names_right.append(answer2)
            self.data["film"][film_number] = answer1
            self.data["name"][film_number] = answer2
            pd.DataFrame(self.list_names_right).to_csv('list_names_right.csv',index=False) 
            pd.DataFrame(self.list_names_wrong).to_csv('list_names_wrong.csv',index=False)
            pd.DataFrame(self.list_films_right).to_csv('list_films_right.csv',index=False) 
            pd.DataFrame(self.list_films_wrong).to_csv('list_films_wrong.csv',index=False) 
        if question == "p":
            pass
       
        
# Alejandro González Iñárritu


In [159]:
test = Oscar_Scraper(1930,2021)
test.getHTML()
test.getDATA()
test.getAPI_omdb()

print(len(test.data["film"]))
print(len(test.data["name"]))
print(len(test.list_gender))
print(len(test.list_birthday))


100%|██████████████████████████████████████████████████████████████████████████| 1372/1372 [00:00<00:00, 338843.85it/s]

1372
1372
1372
1372
